In [0]:
%run ../config/load_config

In [0]:
from pyspark.sql.functions import *

target_table = "stops_geo_sv"
silver_table_path = get_storage_path("silver", target_table)

# Ensure silver table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema_silver}.{target_table}(
    naptan_id STRING,
    stop_name STRING,
    stop_type STRING,
    borough_code STRING,
    borough_name STRING,
    longitude DOUBLE,
    latitude DOUBLE
)
LOCATION '{silver_table_path}'
""")

## Join stop points and london boroughs to derive table (spatial enrichment)
# Read stop_points and create geometry
df_stops = (
    spark.read
        .table(f"{catalog}.{schema_silver}.stops_sv")
        .select(
            "naptan_id",
            col("common_name").alias("stop_name"),"stop_type","longitude","latitude",
            expr("ST_SetSRID(ST_Point(longitude, latitude), 4326)").alias("stop_geom")
        )
)

# Read london_boroughs and create geometry
df_boroughs = (
    spark.read
        .table(f"{catalog}.{schema_silver}.boroughs_sv")
        .select(
            "borough_code",col("borough_name"),
            expr(
                "ST_GeomFromGeoJSON("
                "to_json(named_struct('type','Polygon','coordinates',"
                "from_json(geometry_geojson,"
                "'struct<coordinates:struct<coordinates:array<array<array<double>>>>>')"
                ".coordinates.coordinates)))"
            ).alias("borough_geom")
        )
)

# Spatial join to assign borough to stop
df_silver = (
    df_stops
        .join(broadcast(df_boroughs),expr("ST_Contains(borough_geom, stop_geom)"),"inner")
        .select("naptan_id","stop_name","stop_type","borough_code","borough_name","longitude","latitude")
)

df_silver.createOrReplaceTempView(f"{target_table}_temp_view")

spark.sql(f"""
    MERGE INTO {catalog}.{schema_silver}.{target_table} AS target
    USING {target_table}_temp_view AS source
        ON target.naptan_id = source.naptan_id
    WHEN NOT MATCHED THEN INSERT *
""")